# K-Clustering Tool for Edge Devices

## Table of Contents

### 1. Feature Definition and Data Collection
- Select measurable features suitable for edge devices (size, color, brightness, sensor data).
- Collect data from devices or public datasets.
- Normalize and store data efficiently for limited hardware.

### 2. K-Means Classification on Edge Devices
- Implement a lightweight K-Means algorithm optimized for CPU/GPU constraints.
- Use reduced memory operations and optional medoid-style center updates.
- Compare center initialization methods:
  - Random
  - PCA-based
  - Nearest raw point (K-Medoids approach)
- Final script executes the full pipeline and shows result variation based on initial centers.

### 3. Integration with Split Inference
- Extract features on the edge device to minimize bandwidth.
- Perform clustering locally for real-time classification.
- Offload model retraining to the cloud.
- Synchronize updated cluster centers back to devices with minimal data transfer.
### 4. Resource 
- [StackOverflow](https://stackoverflow.com/questions/35952124/how-to-choose-initial-centroids-for-k-means-clustering)
- [Init centroids smarter](https://medium.com/@atharv4study/k-means-smarter-initialization-for-better-clustering-6914a473e1c7)



### 1. Define feature and collect data 

### 2. Build algorithm 

In [25]:
import math
import random
import numpy as np
import pandas as pd
from typing import List, Tuple


In [26]:
# config 
NUM_CLUSTERS = 2
MAX_ITERATIONS = 500 

In [27]:
data = pd.read_csv("data_device.csv")
raw_points = list(data.values)
raw_points = np.delete(raw_points, 0 , axis=1)

# Manual normalization
points, min_vals, max_vals = min_max_normalize(raw_points)

# CPU & Core slightly emphasized after scaling
FEATURE_INDEX = {
"Total Ram": 0,
"Storage": 1,
"Internet": 2,
"Core": 3
}

WEIGHTS = {
    "Core": 1.4,
    "Total Ram": 1.2,
    "Internet": 1.1,
    "Storage": 0.65
}

points = np.array(points)

for feat, w in WEIGHTS.items():
    points[:, FEATURE_INDEX[feat]] *= w

points = points.tolist()

# Run K-means on full-dimensional data
final_centers, clusters, init_centers = kmeans(
    points=points,
    num_clusters=NUM_CLUSTERS,
    max_iterations=MAX_ITERATIONS
)



#### Normalize

In [28]:
def min_max_normalize(data):
    """
    Perform Min-Max normalization on a dataset without using any external library.

    Args:
        data : list of samples
               shape = (N, D)

    Returns:
        normalized_data : same shape as input, values scaled into [0, 1]
        min_vals        : list[D]
        max_vals        : list[D]
    """
    
    num_features = len(data[0])

    # ---- find min & max for each feature ----
    min_vals = [float("inf")] * num_features
    max_vals = [float("-inf")] * num_features

    for row in data:
        for i, val in enumerate(row):
            if val < min_vals[i]:
                min_vals[i] = val
            if val > max_vals[i]:
                max_vals[i] = val

    # ---- normalize ----
    normalized_data = []
    
    for row in data:
        norm_row = []
        for i, val in enumerate(row):

            # Avoid divide-by-zero if data column is constant
            if max_vals[i] == min_vals[i]:
                scaled = 0.0
            else:
                scaled = (val - min_vals[i]) / (max_vals[i] - min_vals[i])
            
            norm_row.append(scaled)

        normalized_data.append(norm_row)

    return normalized_data, min_vals, max_vals

#### Basic distance

In [29]:
def euclidean_distance(point1: List[float], point2: List[float]) -> float:
    """
    Compute Euclidean distance between two N-D points.

    This implementation avoids numpy heavy operations to reduce RAM usage,
    making it suitable for edge devices.
    """
    s = 0.0
    for x, y in zip(point1, point2):
        s += (x - y) ** 2
    return math.sqrt(s)


def min_distance_to_centroids(point: List[float], centroids: List[List[float]]) -> float:
    """
    Compute the minimum distance from 'point' to all existing centroids.
    Used by the farthest-point initialization heuristic.
    """
    return min(euclidean_distance(point, c) for c in centroids)

#### Farthetst-point centroid initialization 

In [30]:
def select_next_centroid(centroids: List[List[float]],
                          points: List[List[float]]) -> List[float]:
    """
    Select the next centroid using the farthest-point heuristic.

    The next centroid is chosen as the point which has the greatest
    minimum distance to the existing centroid set.

    This technique improves initialization stability over random sampling.
    """
    best_candidate = None
    max_dist = -1

    for p in points:
        dist = min_distance_to_centroids(p, centroids)

        # Avoid duplicate centroid selection
        already_selected = any(np.allclose(p, c) for c in centroids)

        if dist > max_dist and not already_selected:
            max_dist = dist
            best_candidate = p

    return best_candidate

#### Nearest raw data point mapping 

In [31]:
def nearest_point(raw_points: List[Tuple[float, float]],
                  target: Tuple[float, float]) -> Tuple[float, float]:
    """
    Find the real data point closest to a centroid.

    This is important for edge-device deployment where cluster
    representatives must be real measurable samples.
    """
    best = None
    best_dist = float("inf")

    for p in raw_points:
        d = euclidean_distance(p, target)
        if d < best_dist:
            best_dist = d
            best = p

    return best


#### K clustering 

In [32]:
def kmeans(points: List[List[float]],
           num_clusters: int,
           max_iterations: int):
    """
    Perform manual K-Means clustering.

    Steps:
    1. Initialize centroids using random + farthest-point heuristics.
    2. Iteratively assign points to closest centroid.
    3. Update centroids as cluster means.
    4. Stop when centroids converge or iteration limit is reached.
    5. Map centroids to nearest real data points.

    Returns:
        final_centers   : Closest raw points to converged centroids
        clusters        : List of k clusters
        initial_centers : Snapshot of centroid initialization
    """

    # ---- STEP 1: Initialization ----

    # Pick 1 random starting centroid
    centroids = random.sample(points, 1)

    # Use farthest-point heuristic to select remaining centroids
    while len(centroids) < num_clusters:
        new_centroid = select_next_centroid(centroids, points)
        centroids.append(new_centroid)

    initial_centers = centroids.copy()  # For visualization or debugging

    # ---- STEP 2: ITERATIVE REFINEMENT ----

    for iteration in range(max_iterations):

        clusters = [[] for _ in range(num_clusters)]

        # ------ Assignment step -------
        for p in points:
            distances = [euclidean_distance(p, c) for c in centroids]
            cluster_id = distances.index(min(distances))
            clusters[cluster_id].append(p)

        # ------ Update step -------
        new_centroids = []
        for cluster in clusters:

            if not cluster:
                new_centroids.append(random.choice(points))
                continue

            centroid = np.mean(cluster, axis=0).tolist()
            new_centroids.append(centroid)

        # ------ Convergence check -------
        if np.allclose(np.array(new_centroids), np.array(centroids), atol=1e-6):
            print(f"K-Means converged at iteration {iteration}.")
            break

    # ---- STEP 3: Map centroids to nearest real data points ----

    final_centers = [
        nearest_point(points, centroid)
        for centroid in centroids
    ]

    return final_centers, clusters, initial_centers